In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=False,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=False,
    transform=ToTensor()
)

bsize = 64
train_dataloader = DataLoader(training_data, batch_size=bsize)
test_dataloader = DataLoader(test_data, batch_size=bsize)

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super(NeuralNetwork,self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
import torchvision.models as models

In [5]:
model = NeuralNetwork()
model.load_state_dict(torch.load("data/model_1.pth"))
model.eval()

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)

set hypm

In [6]:
learning_rate = 1e-3
batch_size = 64
epochs = 5
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"test error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n----------------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
----------------------------------------
loss: 0.606027 [    0/60000]
loss: 0.675668 [ 6400/60000]
loss: 0.574102 [12800/60000]
loss: 0.820726 [19200/60000]
loss: 0.669711 [25600/60000]
loss: 0.826252 [32000/60000]
loss: 0.659233 [38400/60000]
loss: 0.860077 [44800/60000]
loss: 0.787595 [51200/60000]
loss: 0.777566 [57600/60000]
test error: 
 Accuracy: 76.5%, Avg loss: 0.011320 

Epoch 2
----------------------------------------
loss: 0.601155 [    0/60000]
loss: 0.672516 [ 6400/60000]
loss: 0.570776 [12800/60000]
loss: 0.815864 [19200/60000]
loss: 0.665578 [25600/60000]
loss: 0.823287 [32000/60000]
loss: 0.654921 [38400/60000]
loss: 0.859634 [44800/60000]
loss: 0.785470 [51200/60000]
loss: 0.772806 [57600/60000]
test error: 
 Accuracy: 76.6%, Avg loss: 0.011273 

Epoch 3
----------------------------------------
loss: 0.596671 [    0/60000]
loss: 0.669569 [ 6400/60000]
loss: 0.567952 [12800/60000]
loss: 0.811250 [19200/60000]
loss: 0.661728 [25600/60000]
loss: 0.820144 [32000/60

In [9]:
model_vgg = models.vgg16(pretrained=True)

In [10]:
print(model_vgg)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [11]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n----------------------------------------")
    train_loop(train_dataloader, model_vgg, loss_fn, optimizer)
    test_loop(test_dataloader, model_vgg, loss_fn)
print("Done!")

Epoch 1
----------------------------------------


RuntimeError: Given groups=1, weight of size [64, 3, 3, 3], expected input[64, 1, 28, 28] to have 3 channels, but got 1 channels instead